In [16]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from math import exp
from scipy import stats
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 300)

## Step 1: Read in hold out data, scalers, and best model

In [17]:
kc_house_data_test=pd.read_csv('Data/kc_house_data_test_features.csv', index_col='id').drop('Unnamed: 0', axis=1)
kc_house_data_test.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,
1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [18]:
read_pickle = open('Pickle_Jar/model.pickle', 'rb')
model = pickle.load(read_pickle)
read_pickle.close()

In [19]:
read_pickle = open('Pickle_Jar/column_selection.pickle', 'rb')
column_selection = pickle.load(read_pickle)
read_pickle.close()

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [31]:
len(column_selection)

107

In [20]:
kc_house_data_test['sqft_living^3'] = kc_house_data_test['sqft_living']**3
kc_house_data_test['sqft_living^2'] = kc_house_data_test['sqft_living']**2
kc_house_data_test['sqft_basement^2'] = kc_house_data_test['sqft_basement']**2
kc_house_data_test['yard_space'] = kc_house_data_test.sqft_lot - (kc_house_data_test.sqft_living / kc_house_data_test.floors)
kc_house_data_test['multi_floor'] = [0 if x == 1 else 1 for x in kc_house_data_test.floors] 
kc_house_data_test['has_basement'] = [0 if x == 0 else 1 for x in kc_house_data_test.sqft_basement]
kc_house_data_test['sale_month'] = pd.DatetimeIndex(kc_house_data_test['date']).month
kc_house_data_test['yr_built_reno'] = np.where((kc_house_data_test['yr_built'] < 1974) & (kc_house_data_test['yr_renovated'] == 0), 0,1)
kc_house_data_test

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_living^3,sqft_living^2,sqft_basement^2,yard_space,multi_floor,has_basement,sale_month,yr_built_reno
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,11697083000,5152900,532900,9230.0,0,1,8,0
1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,11697083000,5152900,532900,9230.0,0,1,2,0
3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576,3176523000,2160900,96100,1044.0,1,1,11,1
1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565,2097152000,1638400,62500,14920.0,0,1,12,1
5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916,22665187000,8008900,0,6711.0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263000018,20140521T000000,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509,3581577000,2340900,0,621.0,1,0,5,1
6600060120,20150223T000000,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200,12326391000,5336100,0,4658.0,1,0,2,1
1523300141,20140623T000000,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007,1061208000,1040400,0,840.0,1,0,6,1


In [21]:
kc_house_data_test=pd.get_dummies(kc_house_data_test, columns=['zipcode'], drop_first=True)
kc_house_data_test=pd.get_dummies(kc_house_data_test, columns=['grade'], drop_first=True)
kc_house_data_test=pd.get_dummies(kc_house_data_test, columns=['view'], drop_first=True)
kc_house_data_test['grade_3']=0
kc_house_data_test['biggestzipcode_sqftliving'] = kc_house_data_test['zipcode_98052']*kc_house_data_test['sqft_living']
kc_house_data_test['bestview_sqftliving'] = kc_house_data_test['view_4']*kc_house_data_test['sqft_living']

In [22]:
final_features=kc_house_data_test

In [34]:
len(kc_house_data_test.columns)

110

## Step 3: Predict the holdout set

In [30]:
len(model.coef_)

223

In [27]:
final_answers = model.predict(kc_house_data)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 223 is different from 107)

In [24]:
final_answers

NameError: name 'final_answers' is not defined

## Step 4: Export your predictions

In [22]:
pd.DataFrame(final_answers).to_csv('housing_preds_Jax_theawesome.csv')